<a href="https://colab.research.google.com/github/Deland78/KP_Lead_Scoring_Colab/blob/main/DELeadDataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lead Scoring Raw KP Leads Data Cleansing

In [ ]:
# Cell 1: Install helper package and download the CSV from Google Drive
# If you prefer to mount Drive manually, you can skip gdown and use drive.mount() instead.

!pip install -q gdown  # Install gdown if not already installed

import pandas as pd
import numpy as np

# Google Drive file ID extracted from your shared link
file_id = '1a6x-x3VCA7vm6Gi5T0aCLO_7YJFQKAr9'
# Download the CSV to the Colab environment
!gdown --id $file_id --output 'KP_Lead_Scoring_Data.csv'

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1a6x-x3VCA7vm6Gi5T0aCLO_7YJFQKAr9
To: /content/KP_Lead_Scoring_Data.csv
100% 6.53M/6.53M [00:00<00:00, 242MB/s]


In [ ]:
# Cell 2: Load the dataset and perform initial cleansing
df = pd.read_csv('KP_Lead_Scoring_Data.csv')

# Remove any duplicate rows based on the unique identifier
df = df.drop_duplicates(subset=['OpportunityId'])

# Convert string date columns to datetime
date_cols = ['Created On Datetime', 'Applied Datetime', 'Reinquiry Date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Add an applied_flag (1 if an application date exists, 0 otherwise)
df['applied_flag'] = df['Applied Datetime'].notna().astype(int)

# If Lead to Applied Days is missing but dates exist, compute it
mask = df['Lead to Applied Days'].isna() & df['applied_flag'].eq(1)
df.loc[mask, 'Lead to Applied Days'] = (
    df.loc[mask, 'Applied Datetime'] - df.loc[mask, 'Created On Datetime']
).dt.days


In [ ]:
# Cell 3: Standardize categorical strings
# Strip whitespace from key categorical columns
cat_cols = [
    'Channel','Source','SubSource','Lead Group','Campaign Source Code',
    'When Like To Begin Studying','Highest Level Of Education',
    'Intent To Enroll','Keyword','Postal Code','Opportunity Type','Country'
]
for col in cat_cols:
    df[col] = df[col].astype(str).str.strip()

# Normalize “When Like To Begin Studying” values
study_map = {
    'within 3 months': 'within 3 months',
    '3 months': 'within 3 months',
    'within3months': 'within 3 months',
    'within 6 months': 'within 6 months',
    '6 months': 'within 6 months',
    'within6months': 'within 6 months',
    'within 12 months': 'within 12 months',
    '12 months': 'within 12 months',
    'within12months': 'within 12 months',
    '12 months plus': '12 months plus',
    '12  months plus': '12 months plus',
    '12monthsplus': '12 months plus',
    'unsure': 'unsure',
    ' unsure': 'unsure',
    '9 months': '9 months'
}
df['When Like To Begin Studying'] = df['When Like To Begin Studying'].replace(study_map)

# Normalize “Intent To Enroll” values
df['Intent To Enroll'] = df['Intent To Enroll'].astype(str).str.lower().str.strip()
intent_map = {
    'researchingoptions': 'researching options',
    'researching study options': 'researching options',
    'lookingformoreinformation': 'looking for more information',
    'looking for more info': 'looking for more information',
    'looking for more information': 'looking for more information',
    'looking to get advice': 'need advice or assistance',
    'needadviceorassistance': 'need advice or assistance',
    'need advice or assistance': 'need advice or assistance',
    'ready to enrol': 'ready to enrol',
    'readytoenrol': 'ready to enrol',
    'ready to enroll': 'ready to enrol'
}
df['Intent To Enroll'] = df['Intent To Enroll'].replace(intent_map)

# Ensure Postal Code is stored as string and fill missing with “Unknown”
df['Postal Code'] = df['Postal Code'].replace('nan', np.nan)
df['Postal Code'] = df['Postal Code'].astype(object)
df['Postal Code'] = df['Postal Code'].fillna('Unknown')

# Fill missing categorical values with “Unknown”
cols_to_fill = [
    'Source','SubSource','Lead Group','Campaign Source Code',
    'When Like To Begin Studying','Highest Level Of Education',
    'Intent To Enroll','Keyword','Postal Code','Opportunity Type','Country'
]
for col in cols_to_fill:
    df[col] = df[col].fillna('Unknown')


In [ ]:
# Cell 4: Impute numerical columns and cap outliers

num_cols_to_impute = [
    'Opportunity Count', 'Days Since Created','Days Between Last Opportunities'
]

# Handle Lead to Applied Days -- set to NaN if no application
# If no application, set to NaN
df.loc[df['applied_flag'] == 0, 'Lead to Applied Days'] = np.nan

# Now impute other numeric columns
for col in num_cols_to_impute:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    # Keeping mean imputation as previously attempted, or you can change this
    df[col] = df[col].fillna(df[col].mean())



In [ ]:
# Cell 5: Save the cleaned data back to CSV
cleaned_path = 'DE_Lead_Scoring_Cleaned.csv'
df.to_csv(cleaned_path, index=False)
print(f"Cleaned data saved to {cleaned_path}")

Cleaned data saved to DE_Lead_Scoring_Cleaned.csv
